<a href="https://colab.research.google.com/github/konstanyman/NLP_E13/blob/main/text_generation_pipeline_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [1]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models.

In [3]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

Instantiate a text generation pipeline using the tokenizer and model.

In [4]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [10]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Hyvin, kiitos.”\n”Kiva kuulla.”\n”Kuule, minulla on sinulle asiaa.”\n'}]


Just print the text

In [11]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin, kiitos.”
”Kiva kuulla.”
”Kuule, minulla on sinulle asiaa.”



We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [7]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee?PoistaAnnaR 4 July 2011PostSubsurbedUncategorized Leavea part #1: Nyt on tullut kuunneltua levyjä
4 Rep


In [32]:
sentiment_zero_shot = pipe("Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?\n\nTeksti: Minä olen iloinen.\nVastaus:", max_new_tokens=1)
sentiment_one_shot = pipe("Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?\n\nTeksti: Minä olen iloinen.\nVastaus: Positiivista\n\nTeksti: Minä olen surullinen.\nVastaus:", max_new_tokens=1)
sentiment_two_shot = pipe("Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?\n\nTeksti: Minä olen iloinen.\nVastaus: Positiivista\n\nTeksti: Minä olen surullinen.\nVastaus: Negatiivista\n\nTeksti: Minä olen onnellinen.\nVastaus:", max_new_tokens=1)

In [33]:
print(sentiment_zero_shot[0]['generated_text'])

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Minä olen iloinen.
Vastaus: Kyllä


In [34]:
print(sentiment_one_shot[0]['generated_text'])

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Minä olen iloinen.
Vastaus: Positiivista

Teksti: Minä olen surullinen.
Vastaus: Negatiiv


In [35]:
print(sentiment_two_shot[0]['generated_text'])

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?

Teksti: Minä olen iloinen.
Vastaus: Positiivista

Teksti: Minä olen surullinen.
Vastaus: Negatiivista

Teksti: Minä olen onnellinen.
Vastaus: Positiivista


Kind of capable of recognizing sentiment and answering correctly.

In [53]:
story = "Syksyn hämärtyessä Helsingin kaduilla kulki Pekka, joka haparoi kohti kotiaan.\nHän kohtasi Matiaksen, vanhan ystävänsä, joka tarjoutui saattamaan hänet turvallisesti perille.\nMatiaksen seurassa Pekka tunsi olonsa rauhalliseksi, kunnes heidän tielleen astui salaperäinen\nnainen nimeltä Eveliina, joka toi mukanaan odottamattoman käänteen tarinaan.\n"
name_recognition_zero_shot = pipe(story + "\nNimet:", max_new_tokens=7)
name_recognition_one_shot = pipe(story + "\nNimet:\nPekka", max_new_tokens=5)
name_recognition_two_shot = pipe(story + "\nNimet:\nPekka\nMatias", max_new_tokens=3)

In [54]:
print(name_recognition_zero_shot[0]['generated_text'])

Syksyn hämärtyessä Helsingin kaduilla kulki Pekka, joka haparoi kohti kotiaan.
Hän kohtasi Matiaksen, vanhan ystävänsä, joka tarjoutui saattamaan hänet turvallisesti perille.
Matiaksen seurassa Pekka tunsi olonsa rauhalliseksi, kunnes heidän tielleen astui salaperäinen
nainen nimeltä Eveliina, joka toi mukanaan odottamattoman käänteen tarinaan.

Nimet:
Pekka
Matias
Eveliina


In [55]:
print(name_recognition_one_shot[0]['generated_text'])

Syksyn hämärtyessä Helsingin kaduilla kulki Pekka, joka haparoi kohti kotiaan.
Hän kohtasi Matiaksen, vanhan ystävänsä, joka tarjoutui saattamaan hänet turvallisesti perille.
Matiaksen seurassa Pekka tunsi olonsa rauhalliseksi, kunnes heidän tielleen astui salaperäinen
nainen nimeltä Eveliina, joka toi mukanaan odottamattoman käänteen tarinaan.

Nimet:
Pekka
Matias
Eveliina


In [56]:
print(name_recognition_two_shot[0]['generated_text'])

Syksyn hämärtyessä Helsingin kaduilla kulki Pekka, joka haparoi kohti kotiaan.
Hän kohtasi Matiaksen, vanhan ystävänsä, joka tarjoutui saattamaan hänet turvallisesti perille.
Matiaksen seurassa Pekka tunsi olonsa rauhalliseksi, kunnes heidän tielleen astui salaperäinen
nainen nimeltä Eveliina, joka toi mukanaan odottamattoman käänteen tarinaan.

Nimet:
Pekka
Matias
Eveliina


Nice job!

In [63]:
maths_zero_shot = pipe("Tämä on ensimmäisen luokan matematiikan koe.\n\n13 + 45 =", max_new_tokens=2)
maths_one_shot = pipe("Tämä on ensimmäisen luokan matematiikan koe.\n\n28 + 71 = 99\n13 + 45 =", max_new_tokens=2)
maths_two_shot = pipe("Tämä on ensimmäisen luokan matematiikan koe.\n\n28 + 71 = 99\n16 + 66 = 82\n13 + 45 =", max_new_tokens=2)

In [64]:
print(maths_zero_shot[0]['generated_text'])

Tämä on ensimmäisen luokan matematiikan koe.

13 + 45 = 60



In [65]:
print(maths_one_shot[0]['generated_text'])

Tämä on ensimmäisen luokan matematiikan koe.

28 + 71 = 99
13 + 45 = 72



In [66]:
print(maths_two_shot[0]['generated_text'])

Tämä on ensimmäisen luokan matematiikan koe.

28 + 71 = 99
16 + 66 = 82
13 + 45 = 52



Fail